In [2]:
import nibabel as nib
import numpy as np

dt_path = "/mnt/tempdata/lucas/fmri/recordings/TR/neural/final_all_ses/glasser_resampled/sub-01/ses-01/sub-01_ses-01_task-1back_acq-ctg_run-01_space-Glasser64k_bold.dtseries.nii"
pt_path = "/mnt/tempdata/lucas/fmri/recordings/TR/neural/final_all_ses/glasser_parced/sub-01/ses-01/sub-01_ses-01_task-1back_acq-ctg_run-01_desc-Glasser64k_bold.ptseries.nii"

dt_img = nib.load(dt_path)
pt_img = nib.load(pt_path)

dt_data = dt_img.get_fdata()  # shape: (n_timepoints, n_grayordinates)
pt_data = pt_img.get_fdata()  # shape: (n_parcels, n_timepoints)

print("dtseries shape:", dt_data.shape)
print("ptseries shape:", pt_data.shape)

dtseries shape: (206, 64984)
ptseries shape: (206, 360)


In [8]:
print("dt_data NaNs:", np.isnan(dt_data).sum())
print("pt_data NaNs:", np.isnan(pt_data).sum())
print("Any NaNs?", np.isnan(dt_data).any())
print("Any NaNs?", np.isnan(pt_data).any())

dt_data NaNs: 0
pt_data NaNs: 0
Any NaNs? False
Any NaNs? False


In [9]:
glasser_atlas_str= '/home/lucas/projects/MULTFSCTRL/prep/fmriprep/Glasser_LR_Dense64k.dlabel.nii'
glasser_atlas = nib.load(glasser_atlas_str).get_fdata()[0].astype(int)
print("Glassier Atlas shape:", glasser_atlas.shape)

Glassier Atlas shape: (64984,)


In [10]:
# dt_data: shape (T, N_grayordinates)
# glasser_atlas: shape (N_grayordinates,), values 0..360
# ROI IDs 1..360 (0 is unlabeled)

T, N = dt_data.shape
roi_ids = np.arange(1, 361)   # 1–360

dt_parcel_data = np.zeros((T, 360), dtype=np.float32)

for i, roi in enumerate(roi_ids):
    mask = (glasser_atlas == roi)
    dt_parcel_data[:, i] = dt_data[:, mask].mean(axis=1)

print("Output:", dt_parcel_data.shape)

Output: (206, 360)


In [11]:
print("dt_parcel_data NaNs:", np.isnan(dt_parcel_data).sum())
print("Any NaNs?", np.isnan(dt_parcel_data).any())

dt_parcel_data NaNs: 0
Any NaNs? False


In [27]:
# Ensure pt_data is (T, 360)
if pt_data.shape[0] == 360:
    pt_data_T = pt_data.T
else:
    pt_data_T = pt_data

# dt_parcel_data is already (T, 360)
dt_T = dt_parcel_data

import numpy as np

# dt_parcel_data: (T, 360)
# pt_data: (T, 360) OR (360, T)

T, P = dt_parcel_data.shape

# Ensure pt_data is (T, 360)
if pt_data.shape == (P, T):
    pt_T = pt_data.T
elif pt_data.shape == (T, P):
    pt_T = pt_data
else:
    raise ValueError(f"Unexpected shape for pt_data: {pt_data.shape}")

# Parcels to remove (0-based)
bad_parcels = np.array([119, 299])

# Good parcels
good_parcels = np.setdiff1d(np.arange(P), bad_parcels)

# Slice the arrays
dt_valid = dt_parcel_data[:, good_parcels]   # (T, n_valid)
pt_valid = pt_T[:, good_parcels]             # (T, n_valid)

# Compute correlations parcel-wise
correlations = np.zeros(len(good_parcels))

for i, p in enumerate(good_parcels):
    correlations[i] = np.corrcoef(pt_valid[:, i], dt_valid[:, i])[0, 1]

# Report metrics
print("Number of parcels analyzed:", len(good_parcels))
print("Mean corr:", correlations.mean())
print("Median corr:", np.median(correlations))
print("Min corr:", correlations.min())
print("Max corr:", correlations.max())

Number of parcels analyzed: 358
Mean corr: 1.0
Median corr: 1.0
Min corr: 0.9999999999999998
Max corr: 1.0
